# python语言与计算机科学引论
***
2017/11/25

### 生成器
通过列表生成式，我们可以直接创建一个列表。但是，受到内存限制，列表容量肯定是有限的。而且，创建一个包含100万个元素的列表，不仅占用很大的存储空间，如果我们仅仅需要访问前面几个元素，那后面绝大多数元素占用的空间都白白浪费了。

所以，如果列表元素可以按照某种算法推算出来，那我们是否可以在循环的过程中不断推算出后续的元素呢？这样就不必创建完整的list，从而节省大量的空间。在Python中，这种一边循环一边计算的机制，称为**生成器**：generator。

要创建一个generator，有很多种方法。第一种方法很简单，只要**把一个列表生成式的```[]```改成```()```，就创建了一个generator**：

In [98]:
l = [x * x for x in range(10)]
for n in l:
    print(n)
l[2]

0
1
4
9
16
25
36
49
64
81


4

In [109]:
g = (x * x for x in range(10))
g

<generator object <genexpr> at 0xb373ef5c>

创建```l```和```g```的区别仅在于最外层的```[]```和```()```，L是一个list，而g是一个generator。

我们可以直接打印出list的每一个元素，但我们怎么打印出generator的每一个元素呢？

如果要一个一个打印出来，可以通过```next()```函数获得generator的下一个返回值：

In [3]:
next(g)

0

In [4]:
next(g)

1

In [5]:
next(g)

4

In [6]:
next(g)

9

In [101]:
for _ in range(11):
    print(next(g))

0
1
4
9
16
25
36
49
64
81


StopIteration: 

我们讲过，generator保存的是算法，每次调用```next(g)```，就计算出```g```的下一个元素的值，直到计算到最后一个元素，没有更多的元素时，抛出```StopIteration```的错误。

当然，上面这种不断调用```next(g)```实在是太变态了，**正确的方法是使用```for```循环**，因为generator也是**可迭代对象**：

In [102]:
g = (x * x for x in range(10))
l = [x * x for x in range(10)]
for n in g:
    print(n)

0
1
4
9
16
25
36
49
64
81


所以，我们创建了一个generator后，**基本上永远不会调用```next()```，而是通过```for```循环来迭代它**，并且不需要关心```StopIteration```的错误。

generator非常强大。如果推算的算法比较复杂，用类似列表生成式的```for```循环无法实现的时候，还可以用函数来实现。

比如，著名的斐波拉契数列（Fibonacci），除第一个和第二个数外，任意一个数都可由前两个数相加得到：

1, 1, 2, 3, 5, 8, 13, 21, 34, ...

斐波拉契数列用列表生成式写不出来，但是，用函数把它打印出来却很容易：

In [103]:
def fib(m):
    n, a, b = 0, 0, 1
    while n < m:
        print(b)
        a, b = b, a+b
        n += 1
    return 'done'

上面的函数```fib(N)```可以输出斐波那契数列的前N个数：

In [104]:
fib(10)

1
1
2
3
5
8
13
21
34
55


'done'

仔细观察，可以看出，```fib```函数实际上是定义了斐波拉契数列的推算规则，可以从第一个元素开始，推算出后续任意的元素，这种逻辑其实非常类似generator。

也就是说，上面的函数和generator仅一步之遥。要把fib函数变成generator，只需要把print(b)改为**yield** b就可以了：

In [114]:
def fib(m):
    n, a, b = 0, 0, 1
    while n < m:
        yield b
        a, b = b, a+b
        n += 1
    return 'done'

In [108]:
fib(6)

<generator object fib at 0xb0d6768c>

这里，最难理解的就是generator和函数的执行流程不一样。函数是顺序执行，遇到```return```语句或者最后一行函数语句就返回。而变成generator的函数，在每次调用```next()```的时候执行，遇到```yield```语句返回，再次执行时从上次返回的```yield```语句处继续执行。

举个简单的例子，定义一个generator，依次返回数字1，3，5：

In [36]:
def odd():
    print('step 1')
    yield 1
    print('step 2')
    yield(3)
    print('step 3')
    yield(5)

调用该generator时，首先要生成一个generator对象，然后用```next()```函数不断获得下一个返回值：

In [110]:
o = odd()
next(o)

step 1


1

In [111]:
next(o)

step 2


3

In [112]:
next(o)

step 3


5

In [113]:
next(o)

StopIteration: 

可以看到，```odd```不是普通函数，而是generator，在执行过程中，遇到```yield```就中断，下次又继续执行。执行3次```yield```后，已经没有```yield```可以执行了，所以，第4次调用```next(o)```就报错。

回到```fib```的例子，我们在循环过程中不断调用```yield```，就会不断中断。当然要给循环设置一个条件来退出循环，不然就会产生一个无限数列出来。

同样的，把函数改成generator后，我们基本上从来不会用```next()```来获取下一个返回值，而是直接使用```for```循环来迭代：

In [115]:
for n in fib(6):
    print(n)

1
1
2
3
5
8


但是用```for```循环调用generator时，发现拿不到generator的```return```语句的返回值。如果想要拿到返回值，必须捕获```StopIteration```错误，返回值包含在```StopIteration```的```value```中：

In [58]:
g = fib(6)
while True:
    try:
        x = next(g)
        print('g:', x)
    except StopIteration as e:
        print('Generator return value:', e.value)
        break

g: 1
g: 1
g: 2
g: 3
g: 5
g: 8
Generator return value: done


## Python的函数式编程
函数是Python内建支持的一种封装，我们通过把大段代码拆成函数，通过一层一层的函数调用，就可以**把复杂任务分解成简单的任务**，这种分解可以称之为面向过程的程序设计。函数就是面向过程的程序设计的基本单元。

而函数式编程（请注意多了一个“式”字）——Functional Programming，虽然也可以归结到面向过程的程序设计，但其思想更接近数学计算。

我们首先要搞明白**计算机（Computer）**和**计算（Compute）**的概念。

在计算机的层次上，CPU执行的是加减乘除的指令代码，以及各种条件判断和跳转指令，所以，汇编语言是最贴近计算机的语言。

而计算则指数学意义上的计算，越是抽象的计算，离计算机硬件越远。

对应到编程语言，就是**越低级的语言，越贴近计算机，抽象程度低，执行效率高，比如C语言；越高级的语言，越贴近计算，抽象程度高，执行效率低，比如Lisp语言**。

函数式编程就是一种抽象程度很高的编程范式，纯粹的函数式编程语言编写的函数没有变量，因此，任意一个函数，只要输入是确定的，输出就是确定的，这种纯函数我们称之为**没有副作用**。而允许使用变量的程序设计语言，由于函数内部的变量状态不确定，同样的输入，可能得到不同的输出，因此，这种函数是有副作用的。

函数式编程的一个特点就是，允许把函数本身作为参数传入另一个函数，还允许返回一个函数！

Python对函数式编程提供部分支持。由于Python允许使用变量，因此，**Python不是纯函数式编程语言**。

### 高阶函数
高阶函数英文叫Higher-order function。什么是高阶函数？我们以实际代码为例子，一步一步深入概念。
#### 变量可以指向函数
以Python内置的求绝对值的函数```abs()```为例，调用该函数用以下代码：

In [59]:
abs(-10)

10

但是，如果只写```abs```呢？

In [60]:
abs

<function abs>

可见，```abs(-10)```是函数调用返回值，而```abs```是函数对象本身。

如果把函数本身赋值给变量呢？

In [61]:
f = abs
f

<function abs>

结论：函数本身也可以赋值给变量，即：变量可以指向函数。

**如果一个变量指向了一个函数，可否通过该变量来调用这个函数？**用代码验证一下：

In [62]:
f = abs
f(-10)

10

成功！说明变量f现在已经指向了```abs```函数本身。直接调用```abs()```函数和调用变量```f()```完全相同。

#### 传入函数
既然变量可以指向函数，函数的参数能接收变量，那么一个函数就可以接收另一个函数作为参数，这种函数就称之为**高阶函数**。

一个最简单的高阶函数：

In [63]:
def add(x, y, f):
    return f(x) + f(y)

当我们调用```add(-5, 6, abs)```时，参数```x，y```和```f```分别接收-5，6和```abs```，根据函数定义，我们可以推导计算过程为：
```python
x = -5
y = 6
f = abs
f(x) + f(y) ==> abs(-5) + abs(6) ==> 11
return 11
```

用代码验证一下：

In [116]:
def add(x, y, f):
    return f(x) + f(y)

print(add(-5, 6, abs))

11


### MAP/REDUCE
我们先看**map**。```map()```函数接收两个参数，一个是函数，一个是```Iterable```，**```map```将传入的函数依次作用到序列的每个元素**，并把结果作为新的```Iterator```返回。

举例说明，比如我们有一个函数$f(x)=x^2$，要把这个函数作用在一个list ```[1, 2, 3, 4, 5, 6, 7, 8, 9]```上，就可以用```map()```实现如下：

![](https://cdn.webxueyuan.com/cdn/files/attachments/0013879622109990efbf9d781704b02994ba96765595f56000/0)

现在，我们用Python代码实现：

In [66]:
def f(x):
    return x*x

r = map(f, range(9))
list(r)

[0, 1, 4, 9, 16, 25, 36, 49, 64]

```map()```传入的第一个参数是```f```，即函数对象本身。由于结果r是一个```Iterator```，```Iterator```是**惰性序列**，因此通过```list()```函数让它把整个序列都计算出来并返回一个list。

你可能会想，不需要```map()```函数，写一个循环/列表生成式，也可以计算出结果：

In [67]:
[f(i) for i in range(9)]

[0, 1, 4, 9, 16, 25, 36, 49, 64]

的确可以，但是，从上面的循环代码，能一眼看明白“把f(x)作用在list的每一个元素并把结果生成一个新的list”吗？

所以，map()作为高阶函数，事实上它把运算规则抽象了。

通过它，我们可以写出更精炼且更具可读性的代码。

我们不但可以计算简单的$f(x)=x^2$，还可以计算任意复杂的函数，比如，把这个list所有数字转为字符串：

In [68]:
list(map(str, [1, 2, 3, 4, 5, 6, 7, 8, 9]))

['1', '2', '3', '4', '5', '6', '7', '8', '9']

再看**reduce**的用法。reduce把一个函数作用在一个序列[x1, x2, x3, ...]上，这个函数必须接收两个参数，reduce把结果继续和序列的下一个元素做累积计算，其效果就是：

```python
reduce(f, [x1, x2, x3, x4]) = f(f(f(x1, x2), x3), x4)
```

事实上，```reduce```是上节课讲过的```accumulate```的列表版本（可以参考第6节课讲义）：

* ```reduce```的函数参数对应```accumulate```的```combiner```。
* ```reduce```的序列参数，通过取索引对应```accumulate```的```term```。
* ```reduce```在合并完序列中所有元素会直接停止，所以不需要```null_value```。

比方说对一个序列求和，就可以用reduce实现：

In [70]:
from functools import reduce
def add(x, y):
    return x+y

reduce(add, range(101))

5050

### 匿名函数
当我们在传入函数时，有些时候，不需要显式地定义函数，直接传入匿名函数更方便。

在Python中，对匿名函数提供了有限支持。还是以map()函数为例，计算f(x)=x2时，除了定义一个f(x)的函数外，还可以直接传入匿名函数：

In [71]:
list(map(lambda x: x * x, [1, 2, 3, 4, 5, 6, 7, 8, 9]))

[1, 4, 9, 16, 25, 36, 49, 64, 81]

通过对比可以看出，匿名函数```lambda x: x * x```实际上就是：

In [72]:
def f(x):
    return x * x

**关键字```lambda```表示匿名函数，冒号前面的```x```表示函数参数。**

**匿名函数有个限制，就是只能有一个表达式，不用写```return```，返回值就是该表达式的结果**

用匿名函数有个好处，因为函数没有名字，不必担心函数名冲突。此外，匿名函数也是一个函数对象，也可以把匿名函数赋值给一个变量，再利用变量来调用该函数：

In [73]:
f = lambda x: x * x
f(5)

25

同样，也可以把匿名函数作为返回值返回，比如：

In [117]:
def build(x, y):
    return lambda: x * x + y * y

# 1^2 + 2^2
build(1, 2)()

5

## 面向对象编程
面向对象编程——Object Oriented Programming，简称OOP，是一种程序设计思想。OOP把对象作为程序的基本单元，一个对象包含了数据和操作数据的函数。

**面向过程**的程序设计把计算机程序视为一系列的命令集合，即一组函数的顺序执行。为了简化程序设计，面向过程把函数继续切分为子函数，即把大块函数通过切割成小块函数来降低系统的复杂度。

而**面向对象**的程序设计把计算机程序视为一组对象的集合，而每个对象都可以接收其他对象发过来的消息，并处理这些消息，计算机程序的执行就是一系列消息在各个对象之间传递。

在Python中，所有数据类型都可以视为对象，当然也可以自定义对象。自定义的对象数据类型就是面向对象中的**类（Class）**的概念。

我们以一个例子来说明面向过程和面向对象在程序流程上的不同之处。

假设我们要处理学生的成绩表，为了表示一个学生的成绩，面向过程的程序可以用一个dict表示：

In [75]:
std1 = { 'name': 'Michael', 'score': 98 }
std2 = { 'name': 'Bob', 'score': 81 }

而处理学生成绩可以通过函数实现，比如打印学生的成绩：

In [76]:
def print_score(std):
    print('%s: %s' % (std['name'], std['score']))

如果采用面向对象的程序设计思想，我们首选思考的不是程序的执行流程，而是```Student```这种数据类型应该被视为一个对象，这个对象拥有```name```和```score```这两个属性（Property）。如果要打印一个学生的成绩，首先必须创建出这个学生对应的对象，然后，给对象发一个```print_score```消息，让对象自己把自己的数据打印出来。

In [77]:
class Student(object):

    def __init__(self, name, score):
        self.name = name
        self.score = score

    def print_score(self):
        print('%s: %s' % (self.name, self.score))

给对象发消息实际上就是调用对象对应的关联函数，我们称之为对象的方法（Method）。面向对象的程序写出来就像这样：

In [78]:
bart = Student('Bart Simpson', 59)
lisa = Student('Lisa Simpson', 87)
bart.print_score()
lisa.print_score()

Bart Simpson: 59
Lisa Simpson: 87


面向对象的设计思想是从自然界中来的，因为在自然界中，类（Class）和实例（Instance）的概念是很自然的。Class是一种抽象概念，比如我们定义的Class——Student，是指学生这个概念，而实例（Instance）则是一个个具体的Student，比如，Bart Simpson和Lisa Simpson是两个具体的Student。

所以，面向对象的设计思想是抽象出Class，根据Class创建Instance。

面向对象的抽象程度又比函数要高，因为**一个Class既包含数据，又包含操作数据的方法**。

### 类和实例
面向对象最重要的概念就是**类（Class）**和**实例（Instance）**，必须牢记**类是抽象的模板**，比如Student类，而实例是根据类创建出来的一个个具体的“对象”，每个对象都拥有相同的方法，但各自的数据可能不同。

仍以Student类为例，在Python中，定义类是通过```class```关键字：

In [80]:
class Student(object):
    pass

class后面紧接着是类名，即Student，类名通常是大写开头的单词，紧接着是(object)，表示该类是从哪个类继承下来的。继承的概念我们后面再讲。通常，如果没有合适的继承类，就使用object类，这是所有类最终都会继承的类。

定义好了Student类，就可以根据Student类创建出Student的实例，创建实例是通过类名+()实现的：

In [81]:
bart = Student()
bart

可以看到，变量```bart```指向的就是一个```Student```的实例，后面的```0x10a67a590```是内存地址，每个```object```的地址都不一样，而```Student```本身则是一个类。

可以自由地给一个实例变量绑定属性，比如，给实例```bart```绑定一个```name```属性：

In [82]:
bart.name = 'Bart Simpson'
bart.name

'Bart Simpson'

由于类可以起到模板的作用，因此，可以在创建实例的时候，把一些我们认为必须绑定的属性**强制填写**进去。通过定义一个特殊的```__init__```方法，在创建实例的时候，就把```name```，```score```等属性绑上去：

In [83]:
class Student(object):

    def __init__(self, name, score):
        self.name = name
        self.score = score

**注意：特殊方法“__init__”前后分别有两个下划线！！！**

注意到```__init__```方法的第一个参数永远是```self```，表示创建的实例本身，因此，在```__init__```方法内部，就可以把各种属性绑定到```self```，因为```self```就指向创建的实例本身。

有了```__init__```方法，在创建实例的时候，就不能传入空的参数了，必须传入与```__init__```方法匹配的参数，但```self```不需要传，Python解释器自己会把实例变量传进去：

In [85]:
bart = Student('Bart Simpson', 59)
print(bart.name,
      bart.score)

Bart Simpson 59


和普通的函数相比，在类中定义的函数只有一点不同，就是第一个参数永远是实例变量```self```，并且，调用时，不用传递该参数。除此之外，类的方法和普通函数没有什么区别，所以，你仍然可以用默认参数、可变参数、关键字参数和命名关键字参数。

#### 数据封装
面向对象编程的一个重要特点就是数据封装。在上面的Student类中，每个实例就拥有各自的name和score这些数据。我们可以通过函数来访问这些数据，比如打印一个学生的成绩：

In [87]:
def print_score(std):
    print('%s: %s' % (std.name, std.score))
    
print_score(bart)

Bart Simpson: 59


但是，既然```Student```实例本身就拥有这些数据，要访问这些数据，就没有必要从外面的函数去访问，可以直接在```Student```类的内部定义访问数据的函数，这样，就把“数据”给封装起来了。这些封装数据的函数是和```Student```类本身是关联起来的，我们称之为类的方法：

In [118]:
class Student(object):

    def __init__(self, name, score):
        self.name = name
        self.score = score
        print('succeed')

    def print_score(self):
        print('%s: %s' % (self.name, self.score))

要定义一个方法，除了第一个参数是self外，其他和普通函数一样。要调用一个方法，只需要在实例变量上直接调用，除了self不用传递，其他参数正常传入：

In [92]:
bart = Student('Bart Simpson', 59)
bart.print_score()

Bart Simpson: 59


这样一来，我们从外部看```Student```类，就只需要知道，创建实例需要给出```name```和```score```，而如何打印，都是在```Student```类的内部定义的，这些数据和逻辑被“封装”起来了，调用很容易，但却不用知道内部实现的细节。

封装的另一个好处是可以给```Student```类增加新的方法，比如```get_grade```：
```python
class Student(object):
    ...

    def get_grade(self):
        if self.score >= 90:
            return 'A'
        elif self.score >= 60:
            return 'B'
        else:
            return 'C'
```

### 抽象屏障——有理数的算术运算
有理数集$\mathbb{Q}$是最简单的数域，其对四则运算皆封闭。而且任意有理数$q$，都可以通过分式表示为：

$$q=\frac{n}{d}$$

其中$n$称为分子，$d$称为分母。

假定我们希望做有理数上的算数，希望能做有理数的四则运算，比较两个有理数是否相等，等等。

作为开始，我们假定有了一种从分子和分母构造出有理数的方法。并且，对于一个给定的有理数，我们有方法取得它的分子和分母。它们分别为：

* ```Rat(n ,d)``` 构造一个有理数，其分子为整数```n```，分母为整数```d```。
* ```numer(q)``` 返回有理数```q```的分子。
* ```denom(q)``` 返回有理数```q```的分母。

根据简单的算数知识，我们知道

$$
\frac{n_1}{d_1} + \frac{n_2}{d_2} = \frac{n_1 d_2 + n_2 d_1}{d_1 d_2} \\
\frac{n_1}{d_1} - \frac{n_2}{d_2} = \frac{n_1 d_2 - n_2 d_1}{d_1 d_2} \\
\frac{n_1}{d_1} \cdot \frac{n_2}{d_2} = \frac{n_1 n_2}{d_1 d_2} \\
\frac{n_1/n_2}{d_1/d_2} = \frac{n_1 d_2}{d_1 n_2} \\
\frac{n_1}{d_1} = \frac{n_2}{d_2} \quad if\ and\ only\ if\ n_1d_2 = n_2d_1
$$

In [3]:
class Rat(object):
    
    def __init__(self, n, d):
        self.n = n
        self.d = d
    
    def numer(self):
        return self.n
    
    def denom(self):
        return self.d
    
    def add(self, a):            # self + a equals to q.add(a)
        new_numer = self.n*a.d + a.n*self.d
        new_denom = self.d*a.d
        return Rat(new_numer, new_denom)
    
    def __add__(self, a):
        new_numer = self.n*a.d + a.n*self.d
        new_denom = self.d*a.d
        return Rat(new_numer, new_denom)
    
    def __sub__(self, a):
        new_numer = self.n*a.d - a.n*self.d
        new_denom = self.d*a.d
        return Rat(new_numer, new_denom)
    '''
    __mul__
    __div__
    '''
    
    def __str__(self):
        return '%d/%d' % (self.n, self.d)
    
    __repr__ = __str__
    
q1 = Rat(1, 2)
q2 = Rat(1, 3)

[q1-q2, q1]

[1/6, 1/2]

In [18]:
q1.__class__ == Rat

True